In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

# Load data and model

First load all the CSV files we will need

In [2]:
train = pd.read_csv("../input/nlp-getting-started/train.csv")
test = pd.read_csv("../input/nlp-getting-started/test.csv")
submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")

Create convenient names for the variables we will be using for training and inference.

In [3]:
train_data = train.text.values
train_labels = train.target.values
test_data = test.text.values

In [4]:
%%time
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-large/5'
embed = hub.KerasLayer(module_url, trainable=True, name='USE_embedding')

CPU times: user 47.7 s, sys: 6.52 s, total: 54.2 s
Wall time: 1min 20s


# Train the model

In [5]:
def build_model(embed):
    
    model = Sequential([
        Input(shape=[], dtype=tf.string),
        embed,
        Dense(1, activation='sigmoid')
    ])
    model.compile(Adam(2e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [6]:
model = build_model(embed)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_embedding (KerasLayer)   (None, 512)               147354880 
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 147,355,393
Trainable params: 147,355,393
Non-trainable params: 0
_________________________________________________________________


In [7]:
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

train_history = model.fit(
    train_data, train_labels,
    validation_split=0.25,
    epochs=4,
    callbacks=[checkpoint],
    batch_size=32
)

Train on 5709 samples, validate on 1904 samples
Epoch 1/4
5709/5709 [==============================] - 109s 19ms/sample - loss: 0.5883 - accuracy: 0.7746 - val_loss: 0.4775 - val_accuracy: 0.8178
Epoch 2/4
5709/5709 [==============================] - 35s 6ms/sample - loss: 0.4353 - accuracy: 0.8424 - val_loss: 0.4447 - val_accuracy: 0.8199
Epoch 3/4
5709/5709 [==============================] - 34s 6ms/sample - loss: 0.3829 - accuracy: 0.8770 - val_loss: 0.4493 - val_accuracy: 0.8220
Epoch 4/4
5709/5709 [==============================] - 26s 4ms/sample - loss: 0.3464 - accuracy: 0.9000 - val_loss: 0.4492 - val_accuracy: 0.8209


In [8]:
model.load_weights('model.h5')
test_pred = model.predict(test_data)

In [9]:
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)